<a href="https://colab.research.google.com/github/sergiolucero/notebooks/blob/main/ruta_museos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install folium
# new day, new idea for a map
# 1. retrieve map info from cartodb
# 2. cluster them (first in five groups, then "naturally")
# 3. establish and plot rutas
import folium, random, pandas as pd
import requests

colors=['red','blue','green','orange','yellow']
ZOOM_LEVEL = 12;TILES='cartodbdark_matter'
CARTO_DATABASE = 'museos_galerias'

data = pd.read_csv('http://quant.cl/static/CSVS/museos.csv')  # should use https://github.com/CartoDB/carto-python
data = data.rename(columns={'longitud':'lng','latitud':'lat'}).drop(['the_geom','cartodb_id','precio','horarios'],axis=1)
data = data[['nombre','nick','direccion','lat','lng']]
#.dropna('lat')
data.head()      # STEP1: OK!  now fix all (lat,lng) nans!!

,nombre,nick,direccion,lat,lng
0,Museo Artequin,artequin,Avda Portales 3530,-33.028728,-71.553089
1,Casa Museo La Chascona,chascona,Fernando Márquez de la Plata 0192,-33.420242,-70.619887
2,Posada del Corregidor,NaN,Esmeralda 749,NaN,NaN
3,Galería Plop!,NaN,Merced 349,NaN,NaN
4,Casa Museo Santa Rosa de Apoquindo,santarosa,Avda. Padre Hurtado Sur 1195,-33.408077,-70.571863


In [ ]:
centroid = (data.lat.median(),data.lng.median())
basemap = folium.Map(location=centroid,tiles=TILES,zoom_start=ZOOM_LEVEL)
for ix, dda in data.iterrows():
  icon = folium.Icon(color=colors[random.randint(0,len(colors)-1)])
  if not pd.np.isnan(dda.lat*dda.lng):
    folium.Marker(location=[dda.lat,dda.lng],icon=icon,
                  popup=pd.DataFrame(dda).to_html()).add_to(basemap)
basemap #.save('tcw.html')

Eso se ve bien! Ahora hagamos cuatro clusters.

In [ ]:
from sklearn.cluster import KMeans
kmean = KMeans(n_clusters=15)
lldata = data[['lat','lng']]
lldata = lldata[lldata['lat']>-70]    # REVIEW OUTLIERS!!
vals = lldata.dropna().values
#vals
kmean.fit(vals)
kmean.labels_ #this will give the expected label for each point
kmean.cluster_centers_ #will give cluster centers


array([[-33.44124826, -70.68117694],
       [-33.0287283 , -71.55308909],
       [-33.4711859 , -70.9122802 ],
       [-33.4449877 , -70.62184627],
       [-33.3290959 , -70.730169  ],
       [-33.3976036 , -70.5944464 ],
       [-33.43923007, -70.64212479],
       [-33.51843784, -70.61066572],
       [-33.48666616, -70.69850693],
       [-33.4202424 , -70.6198868 ],
       [-33.45524368, -70.66277028],
       [-33.43883744, -70.65348524],
       [-33.42570607, -70.64595777],
       [-33.4080772 , -70.5718626 ],
       [-33.4144564 , -70.5980108 ]])

In [ ]:
from collections import Counter
Counter(kmean.labels_)
k6 = [ix for ix,val in enumerate(kmean.labels_) if val==6]
d6=data.loc[lldata.iloc[k6].index]
d6


,nombre,nick,direccion,lat,lng
22,Centro Extensión Universidad Católica,NaN,Alameda 390,-33.441441,-70.644491
24,Taller 0316,NaN,Santa Victoria 0316,-33.447321,-70.636202
31,Museo de Artes Visuales,mavi,José Victorino Lastarria 307,-33.437556,-70.642782
44,Museo de Arte Contemporáneo (P. Forestal),NaN,Ismael Valdes Vergara,-33.435183,-70.644208
50,Museo La Merced,NaN,Mac-Iver 341,-33.439006,-70.646847
64,Museo Nacional de Bellas Artes,NaN,Jose Miguel de la Barra 650,-33.435429,-70.643058
74,Museo Violeta Parra,NaN,Vicuña Mackenna 37,-33.438673,-70.637286


In [ ]:
kmap = folium.Map(location=d6.mean().tolist(),zoom_start=15)
locs = d6[['lat','lng']].values.tolist()  # use osm to make ruta
folium.PolyLine(locations=locs).add_to(kmap)
kmap
#+locs

In [ ]:
# Finalmente, debiéramos calcular una ruta óptima (TSP)